In [2]:
import pandas as pd
import numpy as np

np.random.seed(0)
df_oneway = pd.DataFrame({
    'factor': np.repeat(['A', 'B', 'C'], 10),
    'response': np.random.normal(loc=10, scale=2, size=30)
})
df_oneway.to_excel("dummy_oneway.xlsx", index=False)
df_oneway

,factor,response
0,A,13.528105
1,A,10.800314
2,A,11.957476
3,A,14.481786
4,A,13.735116
5,A,8.045444
6,A,11.900177
7,A,9.697286
8,A,9.793562
9,A,10.821197


In [3]:
# One-way ANOVA
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = df_oneway
# Example: response ~ factor
model = ols('response ~ C(factor)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
# display(anova_table)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(factor),6.115680,2.0,0.614627,0.548238,0.548238,ns
Residual,134.328144,27.0,NaN,NaN,NaN,ns


In [4]:
# Extract degrees of freedom and variance (MSE) from ANOVA table
# Get degrees of freedom for error/residual
df_error = anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['df']

# Get Mean Square Error (MSE) for error/residual
mse_error = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['sum_sq'] / anova_table.iloc[-1]['df']

print(f"Degrees of Freedom (Error): {df_error}")
print(f"Mean Square Error (MSE): {mse_error}")

# You can now use df_error and mse_error for LSD, Tukey HSD, etc.

Degrees of Freedom (Error): 27.0
Mean Square Error (MSE): 4.975116460831623


In [5]:
# LSD Value Calculation
from scipy.stats import t

# Suppose n = number of replicates per group, means = group means
n = 10  # adjust as per your data
alpha = 0.05
t_critical = t.ppf(1 - alpha/2, df_error)
lsd = t_critical * (2 * mse_error / n) ** 0.5
print(f"LSD value: {lsd}")

LSD value: 2.046718467751591


In [6]:
# Mean comparison and significance letters using Tukey HSD
from statsmodels.stats.multicomp import MultiComparison

factor_col = 'factor'
response_col = 'response'

# 1. Calculate group means
means = df.groupby(factor_col)[response_col].mean().reset_index()
means.columns = [factor_col, 'Mean']

# 2. Tukey HSD test
mc = MultiComparison(df[response_col], df[factor_col])
tukey_result = mc.tukeyhsd()

# 3. Assign significance letters (robust version)
import numpy as np

def get_significance_letters(tukey_result, group_names):
    n = len(group_names)
    sig_matrix = np.ones((n, n), dtype=bool)
    # Use tukey_result.summary() to get group pairs and reject status
    summary = tukey_result.summary()
    data = summary.data[1:]  # skip header
    group_to_idx = {name: i for i, name in enumerate(group_names)}
    for row in data:
        g1, g2, _, _, _, _, reject = row
        i, j = group_to_idx[g1], group_to_idx[g2]
        sig_matrix[i, j] = not reject
        sig_matrix[j, i] = not reject
    # Assign letters
    letters = [''] * n
    current_letter = 'A'
    assigned = [False] * n
    for i in range(n):
        if not assigned[i]:
            letters[i] += current_letter
            assigned[i] = True
            for j in range(i+1, n):
                if sig_matrix[i, j]:
                    letters[j] += current_letter
                    assigned[j] = True
            current_letter = chr(ord(current_letter) + 1)
    return dict(zip(group_names, letters))

letters_dict = get_significance_letters(tukey_result, tukey_result.groupsunique)
means['Letter'] = means[factor_col].map(letters_dict)

# 4. Display the table
print(means)

  factor       Mean Letter
0      A  11.476046      A
1      B  10.801292      A
2      C  10.379800      A


In [16]:
# np.random.seed(1)
# df_twoway = pd.DataFrame({
#     'factor1': np.repeat(['X', 'Y'], 15),
#     'factor2': np.tile(np.repeat(['M', 'N', 'O'], 5), 2),
#     'response': np.random.normal(loc=20, scale=3, size=30)
# })
# df_twoway.to_excel("dummy_twoway.xlsx", index=False)
# df_twoway.head()
df_twoway = pd.read_excel("D:\\Study\\Study and Extras\\Scientific Work\\Spinach Rehan\\Data Graphs\\Sheet1Tabl.xlsx", sheet_name="Stat Sheet")
df_twoway

,factor1,factor2,SL,RL,Drybiomas,FreshBiomas,Chla,Chlb,TotChl,Chlab,...,GST,NOcont,MG,LOX,Gly1acitity,Gly2acitity,GSHConte,GR,GlyBet,MIT
0,1,1,21.50,14.61,37.00,214.50,2.150,1.650,3.8100,1.302419,...,28.00,28.00,28.00,28.00,28.00,28.00,28.00,28.00,28.00,28.00
1,1,1,21.90,15.01,37.70,217.00,2.210,1.690,3.9100,1.342419,...,28.40,28.40,28.40,28.40,28.40,28.40,28.40,28.40,28.40,28.40
2,1,1,21.20,14.11,36.20,211.50,2.094,1.620,3.6900,1.272419,...,27.70,27.70,27.70,27.70,27.70,27.70,27.70,27.70,27.70,27.70
3,1,2,26.50,17.15,42.00,264.50,2.650,2.150,4.8000,1.232558,...,28.30,28.30,28.30,28.30,28.30,28.30,28.30,28.30,28.30,28.30
4,1,2,27.10,17.75,42.90,269.00,2.720,2.210,4.8900,1.292558,...,28.80,28.80,28.80,28.80,28.80,28.80,28.80,28.80,28.80,28.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,3,8,10.58,6.69,27.16,108.72,1.166,0.566,1.6066,2.006307,...,27.46,27.46,27.46,27.46,27.46,27.46,27.46,27.46,27.46,27.46
77,3,8,9.68,5.79,26.26,99.72,1.076,0.476,1.4960,1.916307,...,26.56,26.56,26.56,26.56,26.56,26.56,26.56,26.56,26.56,26.56
78,3,9,11.22,7.33,25.60,114.22,1.020,0.620,1.7500,1.802139,...,35.60,35.60,35.60,35.60,35.60,35.60,35.60,35.60,35.60,35.60
79,3,9,11.77,7.88,26.15,119.72,1.075,0.675,1.8050,1.857139,...,36.15,36.15,36.15,36.15,36.15,36.15,36.15,36.15,36.15,36.15


In [17]:
# Two-way ANOVA
model = ols('SL ~ C(factor1) + C(factor2) + C(factor1):C(factor2)', data=df_twoway).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(factor1),2759.444936,2.0,5562.123364,2.941317e-63,2.941317e-63,***
C(factor2),334.420610,8.0,168.520185,2.345888e-35,2.345888e-35,***
C(factor1):C(factor2),27.211909,16.0,6.856270,3.671109e-08,3.671109e-08,***
Residual,13.395067,54.0,NaN,NaN,NaN,ns


In [ ]:
# Extract degrees of freedom and variance (MSE) from ANOVA table
# Get degrees of freedom for error/residual
df_error = anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['df']

# Get Mean Square Error (MSE) for error/residual
mse_error = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['sum_sq'] / anova_table.iloc[-1]['df']

print(f"Degrees of Freedom (Error): {df_error}")
print(f"Mean Square Error (MSE): {mse_error}")

# You can now use df_error and mse_error for LSD, Tukey HSD, etc.

Degrees of Freedom (Error): 54.0
Mean Square Error (MSE): 0.24805679012345683


In [20]:
# LSD Value Calculation
from scipy.stats import t

# Suppose n = number of replicates per group, means = group means
n = 3  # adjust as per your data
alpha = 0.05
t_critical = t.ppf(1 - alpha/2, df_error)
lsd = t_critical * (2 * mse_error / n) ** 0.5
print(f"LSD value: {lsd}")

LSD value: 0.8153013458206587


In [22]:
from statsmodels.stats.multicomp import MultiComparison

# Combine the two factors into a single group label
df_twoway['group'] = df_twoway['factor1'].astype(str) + "_" + df_twoway['factor2'].astype(str)

# Calculate means for each group
means_2way = df_twoway.groupby('group')['SL'].mean().reset_index()
means_2way.columns = ['group', 'Mean']

# Tukey HSD test on the interaction groups
mc_2way = MultiComparison(df_twoway['SL'], df_twoway['group'])
tukey_result_2way = mc_2way.tukeyhsd()

# Use the same get_significance_letters function as before
letters_dict_2way = get_significance_letters(tukey_result_2way, tukey_result_2way.groupsunique)
means_2way['Letter'] = means_2way['group'].map(letters_dict_2way)

print(means_2way)

   group       Mean Letter
0    1_1  21.533333      A
1    1_2  26.500000      B
2    1_3  24.600000      C
3    1_4  22.463333      A
4    1_5  24.433333      C
5    1_6  28.600000      D
6    1_7  29.900000      D
7    1_8  25.566667     BC
8    1_9  26.533333      B
9    2_1  14.523333      E
10   2_2  17.616667      F
11   2_3  16.556667      F
12   2_4  15.633333      E
13   2_5  16.283333      F
14   2_6  19.946667      G
15   2_7  20.533333     AG
16   2_8  15.290000      E
17   2_9  15.963333      E
18   3_1   8.466667      H
19   3_2  12.556667      I
20   3_3  11.673333      I
21   3_4   9.866667      H
22   3_5  10.576667      J
23   3_6  13.483333     EI
24   3_7  14.466667      E
25   3_8  10.126667      J
26   3_9  11.253333     IJ


In [8]:
# np.random.seed(2)
# df_threeway = pd.DataFrame({
#     'f1': np.repeat(['P', 'Q'], 12),
#     'f2': np.tile(np.repeat(['R', 'S'], 6), 2),
#     'f3': np.tile(['U', 'V', 'U', 'V', 'U', 'V'], 4),
#     'response': np.random.normal(loc=30, scale=4, size=24)
# })
# df_threeway.to_excel("dummy_threeway.xlsx", index=False)
# df_threeway.head()
df_threeway = pd.read_excel("C:\\Users\\mirza\\OneDrive\\Desktop\\Copy of Mungbean_Excel_sheet(1).xlsx", sheet_name="Sheet3")
df_threeway

,Treatment,Cultivar,Stress,Replicates,RL,SL,RFW,SFW,RDW,SDW,...,TC_content,Caroteniods,Osmotic_Potential_(-Mpa),Relative_Water_Content_(%),Seed_yield/plant(g),No_of_pods_per_plant,Potassium(mg_g-1DW)_,Phosphorus(mg_g-1DW)_,Calcium(mg_g-1DW)_,Magnesium(mg_g-1DW)_
0,1,1,1,1,34.2,35.8,0.66,1.33,0.061,0.33,...,745,28.5,2.00,70.5,3.30,27.0,8625,6.30,4.95,2.20
1,1,1,1,2,34.9,36.5,0.69,1.37,0.065,0.37,...,749,29.0,2.05,71.5,3.34,27.5,8640,6.35,5.00,2.25
2,1,1,1,3,33.5,35.1,0.63,1.29,0.057,0.29,...,741,28.0,1.95,69.5,3.26,26.5,8610,6.25,4.90,2.15
3,1,1,2,1,19.2,19.8,0.40,0.91,0.040,0.16,...,475,19.3,1.60,60.2,2.15,18.0,6705,4.10,3.45,1.50
4,1,1,2,2,19.6,20.5,0.44,0.96,0.046,0.19,...,481,19.9,1.65,61.0,2.20,18.8,6715,4.17,3.49,1.56
5,1,1,2,3,18.8,19.1,0.36,0.86,0.034,0.13,...,469,18.7,1.55,59.4,2.10,17.2,6695,4.03,3.41,1.44
6,1,2,1,1,30.1,31.5,0.45,1.13,0.055,0.27,...,925,30.5,1.80,72.8,3.35,28.0,6825,5.20,4.95,2.30
7,1,2,1,2,30.5,31.8,0.49,1.17,0.059,0.31,...,933,30.9,1.84,73.5,3.42,28.7,6834,5.26,4.99,2.36
8,1,2,1,3,29.7,31.2,0.41,1.09,0.051,0.23,...,917,30.1,1.76,72.1,3.28,27.3,6816,5.14,4.91,2.24
9,1,2,2,1,15.4,14.2,0.25,0.67,0.024,0.15,...,810,16.5,1.59,60.1,2.20,19.0,5220,3.95,4.30,1.75


In [9]:
# Three-way ANOVA
model = ols('SL ~ C(Treatment)*C(Cultivar)*C(Stress)', data=df_threeway).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)
# Extract degrees of freedom and variance (MSE) from ANOVA table
# Get degrees of freedom for error/residual
df_error = anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['df']

# Get Mean Square Error (MSE) for error/residual
mse_error = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['sum_sq'] / anova_table.iloc[-1]['df']

print(f"Degrees of Freedom (Error): {df_error}")
print(f"Mean Square Error (MSE): {mse_error}")

# You can now use df_error and mse_error for LSD, Tukey HSD, etc.
# Extract degrees of freedom and variance (MSE) from ANOVA table
# Get degrees of freedom for error/residual
df_error = anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['df']

# Get Mean Square Error (MSE) for error/residual
mse_error = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['sum_sq'] / anova_table.iloc[-1]['df']

print(f"Degrees of Freedom (Error): {df_error}")
print(f"Mean Square Error (MSE): {mse_error}")

# You can now use df_error and mse_error for LSD, Tukey HSD, etc.
# LSD Value for Three-way ANOVA for Overlapping letters with original relicate data variance LSD 
import numpy as np
from scipy.stats import t

# Combine all factor columns into a single group label for interaction means
df_threeway['group'] = (
    df_threeway['Treatment'].astype(str) + "_" +
    df_threeway['Cultivar'].astype(str) + "_" +
    df_threeway['Stress'].astype(str)
)

# Calculate means and n for each interaction group, keep original order index
means_3way = df_threeway.groupby('group')['SL'].agg(['mean', 'count']).reset_index()
means_3way.columns = ['group', 'Mean', 'n']
means_3way['Original_Index'] = means_3way.index  # 0-based index

# Sort means descending so highest mean gets 'a'
means_3way = means_3way.sort_values('Mean', ascending=False).reset_index(drop=True)

# Recalculate LSD using the actual n for each group (if n varies, use harmonic mean or min(n))
n_eff = means_3way['n'].min()  # conservative, or use harmonic mean if you prefer
alpha = 0.05
t_critical = t.ppf(1 - alpha/2, df_error)
lsd = t_critical * np.sqrt(2 * mse_error / n_eff)
print(f"Recalculated LSD value (using n={n_eff}): {lsd}")

def get_lsd_letters_cld(means_sorted, lsd):
    group_names = means_sorted['group'].tolist()
    means = means_sorted['Mean'].values
    n = len(means)
    letter_sets = [set() for _ in range(n)]
    current_letter = ord('a')
    for i in range(n):
        used_letters = set()
        for j in range(i):
            if abs(means[i] - means[j]) > lsd:
                used_letters |= letter_sets[j]
        while chr(current_letter) in used_letters:
            current_letter += 1
        letter_sets[i].add(chr(current_letter))
        for j in range(i+1, n):
            if abs(means[i] - means[j]) <= lsd:
                letter_sets[j].add(chr(current_letter))
    letters = [''.join(sorted(s)) for s in letter_sets]
    return dict(zip(group_names, letters))

letters_dict_3way = get_lsd_letters_cld(means_3way, lsd)
means_3way['Letter'] = means_3way['group'].map(letters_dict_3way)

# Show original Excel row (0-based), group, mean, n, and letter
print(means_3way[['Original_Index', 'group', 'Mean', 'n', 'Letter']])
# Save the results to an Excel file
import pandas as pd

# Prepare LSD as a DataFrame
lsd_df = pd.DataFrame({
    'LSD_value': [lsd],  # or lsd_means if you use n=1
    'alpha': [alpha],
    't_critical': [t_critical],
    'df_error': [df_error],
    'mse_error': [mse_error]
})

# Export to Excel
with pd.ExcelWriter('3_wayanova_results.xlsx') as writer:
    anova_table.to_excel(writer, sheet_name='ANOVA', index=True)
    lsd_df.to_excel(writer, sheet_name='LSD', index=False)
    means_3way.to_excel(writer, sheet_name='Means_Comparison', index=False)
print("Exported ANOVA, LSD, and mean comparison to anova_results.xlsx")

,sum_sq,df,F,PR(>F),P-value,Signif.
C(Treatment),1648.0275,3.0,1988.570136,1.986210e-36,1.986210e-36,***
C(Cultivar),435.6075,1.0,1576.859729,8.475887e-29,8.475887e-29,***
C(Stress),2146.6875,1.0,7770.814480,8.978249e-40,8.978249e-40,***
C(Treatment):C(Cultivar),22.8375,3.0,27.556561,5.364419e-09,5.364419e-09,***
C(Treatment):C(Stress),111.6075,3.0,134.669683,3.158016e-18,3.158016e-18,***
C(Cultivar):C(Stress),35.7075,1.0,129.257919,8.973406e-13,8.973406e-13,***
C(Treatment):C(Cultivar):C(Stress),37.4175,3.0,45.149321,1.323557e-11,1.323557e-11,***
Residual,8.8400,32.0,NaN,NaN,NaN,ns


Degrees of Freedom (Error): 32.0
Mean Square Error (MSE): 0.2762500000000002
Degrees of Freedom (Error): 32.0
Mean Square Error (MSE): 0.2762500000000002
Recalculated LSD value (using n=3): 0.8741426921559586
    Original_Index  group  Mean  n Letter
0                8  3_1_1  44.4  3      a
1                4  2_1_1  41.9  3      b
2               10  3_2_1  40.7  3      c
3                9  3_1_2  39.3  3      d
4                6  2_2_1  37.6  3      e
5                0  1_1_1  35.8  3      f
6               12  4_1_1  34.3  3      g
7                5  2_1_2  32.6  3      h
8                2  1_2_1  31.5  3      i
9               14  4_2_1  29.4  3      j
10              11  3_2_2  26.5  3      k
11               7  2_2_2  24.7  3      l
12               1  1_1_2  19.8  3      m
13              13  4_1_2  18.1  3      n
14               3  1_2_2  14.2  3      o
15              15  4_2_2  13.4  3      o
Exported ANOVA, LSD, and mean comparison to anova_results.xlsx


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import t
from statsmodels.stats.multicomp import MultiComparison
import re

# Load your data
file_path = "C:\\Users\\mirza\\OneDrive\\Desktop\\Copy of Mungbean_Excel_sheet(1).xlsx"
sheet_name = "Sheet3"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Automatically get factor and response columns
factor_cols = df.columns[:3].tolist()
response_cols = df.columns[3:].tolist()

def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

def get_lsd_letters_cld(means_sorted, lsd):
    group_names = means_sorted['group'].tolist()
    means = means_sorted['Mean'].values
    n = len(means)
    letter_sets = [set() for _ in range(n)]
    current_letter = ord('a')
    for i in range(n):
        used_letters = set()
        for j in range(i):
            if abs(means[i] - means[j]) > lsd:
                used_letters |= letter_sets[j]
        while chr(current_letter) in used_letters:
            current_letter += 1
        letter_sets[i].add(chr(current_letter))
        for j in range(i+1, n):
            if abs(means[i] - means[j]) <= lsd:
                letter_sets[j].add(chr(current_letter))
    letters = [''.join(sorted(s)) for s in letter_sets]
    return dict(zip(group_names, letters))

def safe_sheet_name(name, suffix=""):
    # Remove invalid characters and truncate to 25 chars (to allow suffix)
    name = re.sub(r'[\\/*?:\[\]]', '_', str(name))
    base = name[:25]
    return f"{base}{suffix}"

with pd.ExcelWriter('all_responses_anova_results.xlsx') as writer:
    for idx, response in enumerate(response_cols):
        # Drop NA for this response
        df_sub = df[factor_cols + [response]].dropna()
        # Create group label
        df_sub['group'] = df_sub[factor_cols].astype(str).apply(lambda row: '_'.join(row), axis=1)
        # ANOVA
        formula = f'{response} ~ C({factor_cols[0]})*C({factor_cols[1]})*C({factor_cols[2]})'
        model = ols(formula, data=df_sub).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_table['P-value'] = anova_table['PR(>F)']
        anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
        # Degrees of freedom and MSE
        df_error = anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['df']
        mse_error = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df'] if 'Residual' in anova_table.index else anova_table.iloc[-1]['sum_sq'] / anova_table.iloc[-1]['df']
        # LSD
        means_3way = df_sub.groupby('group')[response].agg(['mean', 'count']).reset_index()
        means_3way.columns = ['group', 'Mean', 'n']
        means_3way['Original_Index'] = means_3way.index
        means_3way = means_3way.sort_values('Mean', ascending=False).reset_index(drop=True)
        n_eff = means_3way['n'].min()
        alpha = 0.05
        t_critical = t.ppf(1 - alpha/2, df_error)
        lsd = t_critical * np.sqrt(2 * mse_error / n_eff)
        # LSD letters
        letters_dict_3way = get_lsd_letters_cld(means_3way, lsd)
        means_3way['Letter'] = means_3way['group'].map(letters_dict_3way)
        # Safe sheet names
        base = safe_sheet_name(response)
        anova_table.to_excel(writer, sheet_name=f'{base}_ANOVA')
        pd.DataFrame({'LSD_value': [lsd], 'alpha': [alpha], 't_critical': [t_critical], 'df_error': [df_error], 'mse_error': [mse_error]}).to_excel(writer, sheet_name=f'{base}_LSD', index=False)
        means_3way.to_excel(writer, sheet_name=f'{base}_Means', index=False)
print("Exported all ANOVA, LSD, and mean comparison results for all responses.")



Exported all ANOVA, LSD, and mean comparison results for all responses.


In [ ]:
# Tukhys HSD

from statsmodels.stats.multicomp import MultiComparison

# Combine all factor columns into a single group label for interaction means
df_threeway['group'] = (
    df_threeway['Treatment'].astype(str) + "_" +
    df_threeway['Cultivar'].astype(str) + "_" +
    df_threeway['Stress'].astype(str)
)

# Calculate means for each interaction group
means_3way = df_threeway.groupby('group')['RL'].mean().reset_index()
means_3way.columns = ['group', 'Mean']

# Sort means descending so highest mean gets 'A'
means_3way = means_3way.sort_values('Mean', ascending=False).reset_index(drop=True)

# Tukey HSD test on the interaction groups
mc_3way = MultiComparison(df_threeway['RL'], df_threeway['group'])
tukey_result_3way = mc_3way.tukeyhsd()

# Assign significance letters, starting from highest mean
import numpy as np

def get_significance_letters_highest_first(tukey_result, means_sorted):
    group_names = means_sorted['group'].tolist()
    n = len(group_names)
    sig_matrix = np.ones((n, n), dtype=bool)
    summary = tukey_result.summary()
    data = summary.data[1:]  # skip header
    group_to_idx = {name: i for i, name in enumerate(group_names)}
    for row in data:
        g1, g2, _, _, _, _, reject = row
        if g1 in group_to_idx and g2 in group_to_idx:
            i, j = group_to_idx[g1], group_to_idx[g2]
            sig_matrix[i, j] = not reject
            sig_matrix[j, i] = not reject
    letters = [''] * n
    current_letter = 'A'
    assigned = [False] * n
    for i in range(n):
        if not assigned[i]:
            letters[i] += current_letter
            assigned[i] = True
            for j in range(i+1, n):
                if sig_matrix[i, j]:
                    letters[j] += current_letter
                    assigned[j] = True
            current_letter = chr(ord(current_letter) + 1)
    return dict(zip(group_names, letters))

letters_dict_3way = get_significance_letters_highest_first(tukey_result_3way, means_3way)
means_3way['Letter'] = means_3way['group'].map(letters_dict_3way)

print(means_3way)

In [ ]:
# LSD Value Calculation for Three-way ANOVA for single letters
import numpy as np
from scipy.stats import t

# Combine all factor columns into a single group label for interaction means
df_threeway['group'] = (
    df_threeway['Treatment'].astype(str) + "_" +
    df_threeway['Cultivar'].astype(str) + "_" +
    df_threeway['Stress'].astype(str)
)

# Calculate means for each interaction group
means_3way = df_threeway.groupby('group')['RL'].mean().reset_index()
means_3way.columns = ['group', 'Mean']

# Sort means descending so highest mean gets 'A'
means_3way = means_3way.sort_values('Mean', ascending=False).reset_index(drop=True)

# Calculate LSD value
# You must have already calculated mse_error and df_error from your ANOVA table
# n = number of replicates per group (adjust as needed)
n = 3
alpha = 0.05
t_critical = t.ppf(1 - alpha/2, df_error)
lsd = t_critical * np.sqrt(2 * mse_error / n)

# Assign significance letters using LSD
def get_lsd_letters(means_sorted, lsd):
    group_names = means_sorted['group'].tolist()
    means = means_sorted['Mean'].values
    n = len(means)
    sig_matrix = np.ones((n, n), dtype=bool)
    # Compare all pairs
    for i in range(n):
        for j in range(i+1, n):
            if abs(means[i] - means[j]) > lsd:
                sig_matrix[i, j] = False
                sig_matrix[j, i] = False
    # Assign letters
    letters = [''] * n
    current_letter = 'A'
    assigned = [False] * n
    for i in range(n):
        if not assigned[i]:
            letters[i] += current_letter
            assigned[i] = True
            for j in range(i+1, n):
                if sig_matrix[i, j]:
                    letters[j] += current_letter
                    assigned[j] = True
            current_letter = chr(ord(current_letter) + 1)
    return dict(zip(group_names, letters))

letters_dict_3way = get_lsd_letters(means_3way, lsd)
means_3way['Letter'] = means_3way['group'].map(letters_dict_3way)

print(means_3way)

In [10]:
np.random.seed(3)
df_fourway = pd.DataFrame({
    'f1': np.repeat(['A', 'B'], 16),
    'f2': np.tile(np.repeat(['C', 'D'], 8), 2),
    'f3': np.tile(['E', 'F'], 16),
    'f4': np.tile(['G', 'H', 'G', 'H'], 8),
    'response': np.random.normal(loc=40, scale=5, size=32)
})
df_fourway.to_excel("dummy_fourway.xlsx", index=False)
df_fourway.head()

,f1,f2,f3,f4,response
0,A,C,E,G,48.943142
1,A,C,F,H,42.182549
2,A,C,E,G,40.482487
3,A,C,F,H,30.682536
4,A,C,E,G,38.613059


In [21]:
# Four-way ANOVA
model = ols('response ~ C(f1)*C(f2)*C(f3)*C(f4)', data=df_fourway).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

c:\Users\mirza\miniconda3\envs\python_eda\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
c:\Users\mirza\miniconda3\envs\python_eda\Lib\site-packages\statsmodels\base\model.py:1917: RuntimeWarning: invalid value encountered in divide
  F /= J
c:\Users\mirza\miniconda3\envs\python_eda\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '


,sum_sq,df,F,PR(>F),P-value,Signif.
C(f1),-3.956427e+00,1.0,-1.529019e-01,1.000000,1.000000,ns
C(f2),8.086125e-01,1.0,3.125000e-02,0.861167,0.861167,ns
C(f3),1.449238e-12,1.0,5.600788e-14,1.000000,1.000000,ns
C(f4),NaN,1.0,NaN,NaN,NaN,ns
C(f1):C(f2),1.214356e+00,1.0,4.693054e-02,0.830324,0.830324,ns
C(f1):C(f3),1.684487e-14,1.0,6.509945e-16,1.000000,1.000000,ns
C(f2):C(f3),3.855323e-13,1.0,1.489945e-14,1.000000,1.000000,ns
C(f1):C(f4),NaN,1.0,NaN,NaN,NaN,ns
C(f2):C(f4),2.021531e-01,1.0,7.812500e-03,0.930301,0.930301,ns
C(f3):C(f4),1.160898e+01,1.0,4.486459e-01,0.509372,0.509372,ns


In [12]:
np.random.seed(4)
df_crd = pd.DataFrame({
    'treatment': np.repeat(['T1', 'T2', 'T3', 'T4'], 8),
    'response': np.random.normal(loc=15, scale=2, size=32)
})
df_crd.to_excel("dummy_crd.xlsx", index=False)
df_crd.head()

,treatment,response
0,T1,15.101123
1,T1,15.999903
2,T1,13.008182
3,T1,16.387197
4,T1,14.163397


In [22]:
# CRD (Completely Randomized Design)
# Same as one-way ANOVA, just use your treatment column
model = ols('response ~ C(treatment)', data=df_crd).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(treatment),7.259031,3.0,0.624862,0.604921,0.604921,ns
Residual,108.425466,28.0,NaN,NaN,NaN,ns


In [14]:
np.random.seed(5)
df_rcbd = pd.DataFrame({
    'treatment': np.tile(['T1', 'T2', 'T3', 'T4'], 5),
    'block': np.repeat(['B1', 'B2', 'B3', 'B4', 'B5'], 4),
    'response': np.random.normal(loc=18, scale=2, size=20)
})
df_rcbd.to_excel("dummy_rcbd.xlsx", index=False)
df_rcbd.head()

,treatment,block,response
0,T1,B1,18.882455
1,T2,B1,17.338260
2,T3,B1,22.861542
3,T4,B1,17.495816
4,T1,B2,18.219220


In [23]:
# RCBD (Randomized Complete Block Design)
model = ols('response ~ C(treatment) + C(block)', data=df_rcbd).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(treatment),18.303359,3.0,1.302872,0.318637,0.318637,ns
C(block),16.547933,4.0,0.883438,0.502629,0.502629,ns
Residual,56.193876,12.0,NaN,NaN,NaN,ns


In [24]:
np.random.seed(6)
df_splitplot = pd.DataFrame({
    'mainplot': np.repeat(['M1', 'M2'], 12),
    'subplot': np.tile(np.repeat(['S1', 'S2'], 3), 4),
    'response': np.random.normal(loc=22, scale=3, size=24)
})
df_splitplot.to_excel("dummy_splitplot.xlsx", index=False)
df_splitplot.head()

,mainplot,subplot,response
0,M1,S1,21.064649
1,M1,S1,24.187012
2,M1,S1,22.653462
3,M1,S2,19.302725
4,M1,S2,14.539658


In [25]:
# Split-plot (basic)
model = ols('response ~ C(mainplot) * C(subplot)', data=df_splitplot).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(mainplot),0.152362,1.0,0.011871,0.914325,0.914325,ns
C(subplot),7.020302,1.0,0.546970,0.468151,0.468151,ns
C(mainplot):C(subplot),0.552060,1.0,0.043012,0.837799,0.837799,ns
Residual,256.698028,20.0,NaN,NaN,NaN,ns


In [26]:
np.random.seed(7)
df_splitsplit = pd.DataFrame({
    'mainplot': np.repeat(['M1', 'M2'], 8),
    'subplot': np.tile(np.repeat(['S1', 'S2'], 2), 4),
    'subsubplot': np.tile(['SS1', 'SS2'], 8),
    'response': np.random.normal(loc=25, scale=3, size=16)
})
df_splitsplit.to_excel("dummy_splitsplit.xlsx", index=False)
df_splitsplit.head()

,mainplot,subplot,subsubplot,response
0,M1,S1,SS1,30.071577
1,M1,S1,SS2,23.602188
2,M1,S2,SS1,25.098460
3,M1,S2,SS2,26.222549
4,M1,S1,SS1,22.633231


In [27]:
# Split-split plot (basic)
model = ols('response ~ C(mainplot) * C(subplot) * C(subsubplot)', data=df_splitsplit).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(mainplot),0.034228,1.0,0.004397,0.948756,0.948756,ns
C(subplot),20.986086,1.0,2.696175,0.139216,0.139216,ns
C(subsubplot),12.346646,1.0,1.586228,0.243367,0.243367,ns
C(mainplot):C(subplot),3.808522,1.0,0.489298,0.504068,0.504068,ns
C(mainplot):C(subsubplot),0.363777,1.0,0.046736,0.834254,0.834254,ns
C(subplot):C(subsubplot),0.096196,1.0,0.012359,0.914220,0.914220,ns
C(mainplot):C(subplot):C(subsubplot),0.109330,1.0,0.014046,0.908581,0.908581,ns
Residual,62.269200,8.0,NaN,NaN,NaN,ns


In [28]:
np.random.seed(8)
df_latin = pd.DataFrame({
    'row': np.tile(['R1', 'R2', 'R3', 'R4'], 4),
    'column': np.repeat(['C1', 'C2', 'C3', 'C4'], 4),
    'treatment': np.tile(['T1', 'T2', 'T3', 'T4'], 4),
    'response': np.random.normal(loc=28, scale=2, size=16)
})
df_latin.to_excel("dummy_latin_square.xlsx", index=False)
df_latin.head()

,row,column,treatment,response
0,R1,C1,T1,28.182409
1,R2,C1,T2,30.182565
2,R3,C1,T3,24.106059
3,R4,C1,T4,25.227301
4,R1,C2,T1,23.407017


In [30]:
# Latin Square
model = ols('response ~ C(row) + C(column) + C(treatment)', data=df_latin).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(row),27.389962,3.0,1.074450,0.407685,0.407685,ns
C(column),35.491171,3.0,1.392244,0.307109,0.307109,ns
C(treatment),27.389962,3.0,1.074450,0.407685,0.407685,ns
Residual,76.476206,9.0,NaN,NaN,NaN,ns


In [31]:
np.random.seed(9)
df_splitblock = pd.DataFrame({
    'block1': np.repeat(['B1', 'B2'], 8),
    'block2': np.tile(['B3', 'B4'], 8),
    'treatment': np.tile(['T1', 'T2'], 8),
    'response': np.random.normal(loc=32, scale=2, size=16)
})
df_splitblock.to_excel("dummy_splitblock.xlsx", index=False)
df_splitblock.head()

,block1,block2,treatment,response
0,B1,B3,T1,32.002217
1,B1,B4,T2,31.420912
2,B1,B3,T1,29.767867
3,B1,B4,T2,31.974234
4,B1,B3,T1,31.243277


In [32]:
# Split block (basic)
model = ols('response ~ C(block1) + C(block2) + C(treatment)', data=df_splitblock).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(block1),20.386289,1.0,9.447687,0.008885,0.008885,**
C(block2),0.527040,1.0,0.244248,0.629400,0.629400,ns
C(treatment),0.527040,1.0,0.244248,0.629400,0.629400,ns
Residual,28.051494,13.0,NaN,NaN,NaN,ns


In [33]:
np.random.seed(10)
df_stripsplit = pd.DataFrame({
    'strip1': np.repeat(['S1', 'S2'], 8),
    'strip2': np.tile(['S3', 'S4'], 8),
    'subplot': np.tile(['SP1', 'SP2'], 8),
    'response': np.random.normal(loc=35, scale=2, size=16)
})
df_stripsplit.to_excel("dummy_stripsplit.xlsx", index=False)
df_stripsplit.head()

,strip1,strip2,subplot,response
0,S1,S3,SP1,37.663173
1,S1,S4,SP2,36.430558
2,S1,S3,SP1,31.909199
3,S1,S4,SP2,34.983232
4,S1,S3,SP1,36.242672


In [35]:
# Strip-split (basic)
model = ols('response ~ C(strip1) * C(strip2) * C(subplot)', data=df_stripsplit).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(strip1),2.853806e+00,1.0,1.089007e+00,0.317259,0.317259,ns
C(strip2),4.996966e-13,1.0,1.906832e-13,1.000000,1.000000,ns
C(subplot),3.971065e-14,1.0,1.515350e-14,1.000000,1.000000,ns
C(strip1):C(strip2),1.493571e-13,1.0,5.699436e-14,1.000000,1.000000,ns
C(strip1):C(subplot),1.493571e-13,1.0,5.699436e-14,1.000000,1.000000,ns
C(strip2):C(subplot),1.235756e+00,1.0,4.715620e-01,0.505323,0.505323,ns
C(strip1):C(strip2):C(subplot),2.853806e+00,1.0,1.089007e+00,0.317259,0.317259,ns
Residual,3.144670e+01,12.0,NaN,NaN,NaN,ns


In [36]:
np.random.seed(11)
df_splsqr = pd.DataFrame({
    'row': np.tile(['R1', 'R2', 'R3'], 6),
    'column': np.repeat(['C1', 'C2', 'C3'], 6),
    'mainplot': np.tile(['M1', 'M2'], 9),
    'subplot': np.tile(['S1', 'S2', 'S3'], 6),
    'response': np.random.normal(loc=38, scale=2, size=18)
})
df_splsqr.to_excel("dummy_splitplot_latin_square.xlsx", index=False)
df_splsqr.head()

,row,column,mainplot,subplot,response
0,R1,C1,M1,S1,41.498909
1,R2,C1,M2,S2,37.427854
2,R3,C1,M1,S3,37.030870
3,R1,C1,M2,S1,32.693363
4,R2,C1,M1,S2,37.983431


In [38]:
# Split-plot Latin Square (basic)
model = ols('response ~ C(row) + C(column) + C(mainplot) * C(subplot)', data=df_splsqr).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
def significance_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    elif p < 0.1:
        return '.'
    else:
        return 'ns'

anova_table['P-value'] = anova_table['PR(>F)']
anova_table['Signif.'] = anova_table['PR(>F)'].apply(significance_stars)
display(anova_table)

,sum_sq,df,F,PR(>F),P-value,Signif.
C(row),7.295559,2.0,1.029161,0.392252,0.392252,ns
C(column),4.718326,2.0,0.665599,0.535329,0.535329,ns
C(mainplot),3.284251,1.0,0.926597,0.358447,0.358447,ns
C(subplot),3.977385,2.0,0.561077,0.587565,0.587565,ns
C(mainplot):C(subplot),30.459042,2.0,4.296758,0.044998,0.044998,*
Residual,35.444216,10.0,NaN,NaN,NaN,ns


In [ ]:
# Export ANOVA table to text
anova_table.to_csv("anova_table.txt", sep="\t")

# Export to docx
from docx import Document
doc = Document()
doc.add_heading('ANOVA Table', 0)
t = doc.add_table(rows=1, cols=len(anova_table.columns)+1)
hdr_cells = t.rows[0].cells
hdr_cells[0].text = 'Source'
for i, col in enumerate(anova_table.columns):
    hdr_cells[i+1].text = col
for idx, row in anova_table.iterrows():
    row_cells = t.add_row().cells
    row_cells[0].text = str(idx)
    for i, val in enumerate(row):
        row_cells[i+1].text = str(val)
doc.save("anova_table.docx")

In [ ]:
# Export to PDF
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
fig, ax = plt.subplots(figsize=(8,2))
ax.axis('off')
tbl = ax.table(cellText=anova_table.values, colLabels=anova_table.columns, rowLabels=anova_table.index, loc='center')
plt.tight_layout()
pdf = PdfPages("anova_table.pdf")
pdf.savefig(fig)
pdf.close()

In [ ]:
# Export to XML
import xml.etree.ElementTree as ET
root = ET.Element("ANOVATable")
for idx, row in anova_table.iterrows():
    entry = ET.SubElement(root, "Row", source=str(idx))
    for col, val in row.items():
        ET.SubElement(entry, col).text = str(val)
tree = ET.ElementTree(root)
tree.write("anova_table.xml")